In [2]:
import lyricsgenius as genius
import os
import json
from tqdm import tqdm
import matplotlib.pylab as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

# lol don't use my API key
geniusCreds = 'Lw6NjXtbU7NndUFHRCcOX9FdLhPzVokLIt9c4LWzsTxM10wF7EICGtWSSso8Ohsq'
# for more data, you can adjust the number of artists as well as the max number of songs
artist_names = [
    'James Taylor',
    'The Beatles',
    'Johnny Cash',
    'Van Morrison',
    'Bob Dylan',
    'Aretha Franklin',
    'Fleetwood Mac',
    'J. Cole'
]

api = genius.Genius(geniusCreds)

Using TensorFlow backend.


In [ ]:
artist = api.search_artist('J. Cole', max_songs=100)
artist.save_lyrics()

In [ ]:
for artist_name in artist_names:
    artist = api.search_artist(artist_name, max_songs=100)
    artist.save_lyrics()

In [4]:
for artist in tqdm(artist_names):
    a_mod = artist.replace(' ', '')
    filename = f'Lyrics_{a_mod}.json'
    with open(f'./data/{filename}') as f:
        data = json.load(f)
        song_obj = {}
        song_obj[artist] = []
        for song in data['songs']:
            song_obj[artist].append(song['lyrics'])
        with open(f'./data/cleaned/{artist}.json', 'w') as outfile:
            json.dump(song_obj, outfile)

100%|██████████| 8/8 [00:00<00:00, 43.67it/s]


### Neural Net Stuff

In [71]:
# get basic X, Y sets for training + testing
# X is song lyrics
# Y is the one-hot encoding of the artists
X = []
Y = []

index = 0
for filename in os.listdir('./data/cleaned'):
    with open(f'./data/cleaned/{filename}') as file:
        data = json.load(file)[filename.replace('.json', '')]
        if len(data) == 100:
            for song in data:
                X.append(song)
                # one hot encode artist
                one_h = np.zeros(5)
                one_h[index] = 1
                Y.append(one_h)
            index += 1

In [73]:
# create encoding of words
# i.e. map words to numbers
big_word_list = {}
for song in X:
    for word in song.split(' '):
        if word not in big_word_list.keys():
            if (len(big_word_list.keys()) == 0):
                big_word_list[word] = 1
            else:
                big_word_list[word] = len(big_word_list) + 1

In [74]:
# tokenize the songs
# so we are mapping each word to it's key in the big_word_list
token_x = []
for song in X:
    new_song = []
    for word in song.split(' '):
        new_song.append(big_word_list[word])
    token_x.append(np.array(new_song))
X = token_x

In [75]:
# cutting off lyrics min 100 words
# to normalize network input
cutoff = 100
index = 0
new_x = []
new_y = []
for song in X:
    if (len(song) >= cutoff):
        new_x.append(song)
        new_y.append(Y[index])
    index += 1

X = new_x
Y = new_y

472
472


In [76]:
# trim all of the lyrics to 100 words max,
# so the network input in normalized
trimmed = []
for song in X:
    song = song[:100]
    trimmed.append(song)
X = trimmed

In [79]:
# convert to numpy arrays for keras
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

(472, 100)
(472, 5)


### Create model, train and evaluate

In [92]:
model = Sequential([
    Dense(128, input_shape=(100,)),
    Activation('softmax'),
    Dense(1500),
    Activation('softmax'),
    Dense(500),
    Activation('relu'),
    Dense(225),
    Activation('relu'),
    Dense(64),
    Activation('softmax'),
    Dense(5)
])

model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

In [93]:
model.fit(X, Y, epochs=1000, batch_size=24)

Epoch 1/1000
472/472 [==============================] - 0s 916us/step - loss: 0.1835 - accuracy: 0.1822
Epoch 2/1000
472/472 [==============================] - 0s 538us/step - loss: 0.1625 - accuracy: 0.1843
Epoch 3/1000
472/472 [==============================] - 0s 522us/step - loss: 0.1605 - accuracy: 0.1610
Epoch 4/1000
472/472 [==============================] - 0s 512us/step - loss: 0.1603 - accuracy: 0.1992
Epoch 5/1000
472/472 [==============================] - 0s 512us/step - loss: 0.1603 - accuracy: 0.1843
Epoch 6/1000
472/472 [==============================] - 0s 506us/step - loss: 0.1603 - accuracy: 0.1864
Epoch 7/1000
472/472 [==============================] - 0s 510us/step - loss: 0.1603 - accuracy: 0.1970
Epoch 8/1000
472/472 [==============================] - 0s 512us/step - loss: 0.1603 - accuracy: 0.1822
Epoch 9/1000
472/472 [==============================] - 0s 517us/step - loss: 0.1602 - accuracy: 0.1949
Epoch 10/1000
472/472 [==============================] - 0s 512u

472/472 [==============================] - 0s 535us/step - loss: 0.1601 - accuracy: 0.2161
Epoch 80/1000
472/472 [==============================] - 0s 517us/step - loss: 0.1601 - accuracy: 0.2119
Epoch 81/1000
472/472 [==============================] - 0s 529us/step - loss: 0.1601 - accuracy: 0.2076
Epoch 82/1000
472/472 [==============================] - 0s 527us/step - loss: 0.1600 - accuracy: 0.2076
Epoch 83/1000
472/472 [==============================] - 0s 508us/step - loss: 0.1601 - accuracy: 0.2140
Epoch 84/1000
472/472 [==============================] - 0s 515us/step - loss: 0.1600 - accuracy: 0.2161
Epoch 85/1000
472/472 [==============================] - 0s 535us/step - loss: 0.1599 - accuracy: 0.2076
Epoch 86/1000
472/472 [==============================] - 0s 540us/step - loss: 0.1599 - accuracy: 0.2013
Epoch 87/1000
472/472 [==============================] - 0s 548us/step - loss: 0.1599 - accuracy: 0.2267
Epoch 88/1000
472/472 [==============================] - 0s 533us/ste

472/472 [==============================] - 0s 504us/step - loss: 0.1557 - accuracy: 0.2669
Epoch 157/1000
472/472 [==============================] - 0s 496us/step - loss: 0.1557 - accuracy: 0.2775
Epoch 158/1000
472/472 [==============================] - 0s 509us/step - loss: 0.1554 - accuracy: 0.2691
Epoch 159/1000
472/472 [==============================] - 0s 504us/step - loss: 0.1556 - accuracy: 0.2818
Epoch 160/1000
472/472 [==============================] - 0s 551us/step - loss: 0.1554 - accuracy: 0.2606
Epoch 161/1000
472/472 [==============================] - 0s 644us/step - loss: 0.1554 - accuracy: 0.2733
Epoch 162/1000
472/472 [==============================] - 0s 542us/step - loss: 0.1557 - accuracy: 0.2585
Epoch 163/1000
472/472 [==============================] - 0s 563us/step - loss: 0.1554 - accuracy: 0.2818
Epoch 164/1000
472/472 [==============================] - 0s 584us/step - loss: 0.1555 - accuracy: 0.2966
Epoch 165/1000
472/472 [==============================] - 0s 

472/472 [==============================] - 0s 538us/step - loss: 0.1509 - accuracy: 0.3369
Epoch 234/1000
472/472 [==============================] - 0s 640us/step - loss: 0.1514 - accuracy: 0.3284
Epoch 235/1000
472/472 [==============================] - 0s 569us/step - loss: 0.1513 - accuracy: 0.3326
Epoch 236/1000
472/472 [==============================] - 0s 579us/step - loss: 0.1508 - accuracy: 0.3432
Epoch 237/1000
472/472 [==============================] - 0s 534us/step - loss: 0.1508 - accuracy: 0.3199
Epoch 238/1000
472/472 [==============================] - 0s 538us/step - loss: 0.1511 - accuracy: 0.3284
Epoch 239/1000
472/472 [==============================] - 0s 538us/step - loss: 0.1507 - accuracy: 0.3390
Epoch 240/1000
472/472 [==============================] - 0s 541us/step - loss: 0.1502 - accuracy: 0.3411
Epoch 241/1000
472/472 [==============================] - 0s 540us/step - loss: 0.1514 - accuracy: 0.3199
Epoch 242/1000
472/472 [==============================] - 0s 

472/472 [==============================] - 0s 531us/step - loss: 0.1496 - accuracy: 0.3263
Epoch 311/1000
472/472 [==============================] - 0s 539us/step - loss: 0.1488 - accuracy: 0.3411
Epoch 312/1000
472/472 [==============================] - 0s 485us/step - loss: 0.1491 - accuracy: 0.3432
Epoch 313/1000
472/472 [==============================] - 0s 476us/step - loss: 0.1493 - accuracy: 0.3305
Epoch 314/1000
472/472 [==============================] - 0s 458us/step - loss: 0.1488 - accuracy: 0.3559
Epoch 315/1000
472/472 [==============================] - 0s 469us/step - loss: 0.1493 - accuracy: 0.3305
Epoch 316/1000
472/472 [==============================] - 0s 468us/step - loss: 0.1489 - accuracy: 0.3390
Epoch 317/1000
472/472 [==============================] - 0s 526us/step - loss: 0.1492 - accuracy: 0.3305
Epoch 318/1000
472/472 [==============================] - 0s 526us/step - loss: 0.1487 - accuracy: 0.3347
Epoch 319/1000
472/472 [==============================] - 0s 

...

472/472 [==============================] - 0s 523us/step - loss: 0.1473 - accuracy: 0.3220
Epoch 619/1000
472/472 [==============================] - 0s 503us/step - loss: 0.1472 - accuracy: 0.3347
Epoch 620/1000
472/472 [==============================] - 0s 509us/step - loss: 0.1475 - accuracy: 0.3496
Epoch 621/1000
472/472 [==============================] - 0s 499us/step - loss: 0.1477 - accuracy: 0.3390
Epoch 622/1000
472/472 [==============================] - 0s 500us/step - loss: 0.1468 - accuracy: 0.3432
Epoch 623/1000
472/472 [==============================] - 0s 513us/step - loss: 0.1472 - accuracy: 0.3199
Epoch 624/1000
472/472 [==============================] - 0s 507us/step - loss: 0.1469 - accuracy: 0.3496
Epoch 625/1000
472/472 [==============================] - 0s 515us/step - loss: 0.1472 - accuracy: 0.3369
Epoch 626/1000
472/472 [==============================] - 0s 507us/step - loss: 0.1476 - accuracy: 0.3390
Epoch 627/1000
472/472 [==============================] - 0s 

472/472 [==============================] - 0s 454us/step - loss: 0.1468 - accuracy: 0.3390
Epoch 696/1000
472/472 [==============================] - 0s 473us/step - loss: 0.1466 - accuracy: 0.3411
Epoch 697/1000
472/472 [==============================] - 0s 482us/step - loss: 0.1471 - accuracy: 0.3644
Epoch 698/1000
472/472 [==============================] - 0s 456us/step - loss: 0.1467 - accuracy: 0.3411
Epoch 699/1000
472/472 [==============================] - 0s 454us/step - loss: 0.1475 - accuracy: 0.3390
Epoch 700/1000
472/472 [==============================] - 0s 454us/step - loss: 0.1465 - accuracy: 0.3390
Epoch 701/1000
472/472 [==============================] - 0s 454us/step - loss: 0.1466 - accuracy: 0.3581
Epoch 702/1000
472/472 [==============================] - 0s 451us/step - loss: 0.1468 - accuracy: 0.3538
Epoch 703/1000
472/472 [==============================] - 0s 448us/step - loss: 0.1470 - accuracy: 0.3538
Epoch 704/1000
472/472 [==============================] - 0s 

472/472 [==============================] - 0s 515us/step - loss: 0.1471 - accuracy: 0.3602
Epoch 773/1000
472/472 [==============================] - 0s 506us/step - loss: 0.1463 - accuracy: 0.3496
Epoch 774/1000
472/472 [==============================] - 0s 501us/step - loss: 0.1462 - accuracy: 0.3475
Epoch 775/1000
472/472 [==============================] - 0s 511us/step - loss: 0.1472 - accuracy: 0.3475
Epoch 776/1000
472/472 [==============================] - 0s 507us/step - loss: 0.1465 - accuracy: 0.3559
Epoch 777/1000
472/472 [==============================] - 0s 508us/step - loss: 0.1468 - accuracy: 0.3581
Epoch 778/1000
472/472 [==============================] - 0s 520us/step - loss: 0.1468 - accuracy: 0.3411
Epoch 779/1000
472/472 [==============================] - 0s 501us/step - loss: 0.1463 - accuracy: 0.3623
Epoch 780/1000
472/472 [==============================] - 0s 513us/step - loss: 0.1468 - accuracy: 0.3623
Epoch 781/1000
472/472 [==============================] - 0s 

472/472 [==============================] - 0s 551us/step - loss: 0.1462 - accuracy: 0.3665
Epoch 850/1000
472/472 [==============================] - 0s 565us/step - loss: 0.1457 - accuracy: 0.3665
Epoch 851/1000
472/472 [==============================] - 0s 550us/step - loss: 0.1465 - accuracy: 0.3602
Epoch 852/1000
472/472 [==============================] - 0s 548us/step - loss: 0.1463 - accuracy: 0.3581
Epoch 853/1000
472/472 [==============================] - 0s 565us/step - loss: 0.1462 - accuracy: 0.3538
Epoch 854/1000
472/472 [==============================] - 0s 553us/step - loss: 0.1461 - accuracy: 0.3602
Epoch 855/1000
472/472 [==============================] - 0s 583us/step - loss: 0.1458 - accuracy: 0.3750
Epoch 856/1000
472/472 [==============================] - 0s 554us/step - loss: 0.1464 - accuracy: 0.3644
Epoch 857/1000
472/472 [==============================] - 0s 527us/step - loss: 0.1454 - accuracy: 0.3708
Epoch 858/1000
472/472 [==============================] - 0s 

472/472 [==============================] - 0s 524us/step - loss: 0.1449 - accuracy: 0.3623
Epoch 927/1000
472/472 [==============================] - 0s 516us/step - loss: 0.1453 - accuracy: 0.3665
Epoch 928/1000
472/472 [==============================] - 0s 510us/step - loss: 0.1447 - accuracy: 0.3665
Epoch 929/1000
472/472 [==============================] - 0s 508us/step - loss: 0.1446 - accuracy: 0.3792
Epoch 930/1000
472/472 [==============================] - 0s 557us/step - loss: 0.1451 - accuracy: 0.3602
Epoch 931/1000
472/472 [==============================] - 0s 519us/step - loss: 0.1448 - accuracy: 0.3581
Epoch 932/1000
472/472 [==============================] - 0s 543us/step - loss: 0.1447 - accuracy: 0.3750
Epoch 933/1000
472/472 [==============================] - 0s 457us/step - loss: 0.1448 - accuracy: 0.3517
Epoch 934/1000
472/472 [==============================] - 0s 456us/step - loss: 0.1446 - accuracy: 0.3792
Epoch 935/1000
472/472 [==============================] - 0s 

In [94]:
model.evaluate(X, Y)

472/472 [==============================] - 0s 141us/step


[0.14238506707094484, 0.38347458839416504]